In [21]:
import pandas as pd
import numpy as np
import datetime 
import json

In [22]:
def is_date(strdate):
    '''判断是否是一个有效的日期字符串'''
    try:
        datetime.datetime.strptime(strdate, "%m/%d/%Y, %H:%M:%S")
        return True
    except:
        return False

In [23]:
c_t = "http://www.w3.org/2000/01/rdf-schema#Class"
p_t = "http://www.w3.org/1999/02/22-rdf-syntax-ns#Property"
c_xml = "http://www.openkg.cn/COVID-19/wiki/class/"
p_xml = "http://www.openkg.cn/COVID-19/wiki/property/"
r_xml = "http://www.openkg.cn/COVID-19/wiki/resource/"

path = r"data/百科.json"

In [24]:
'''创建Map'''
class_map = {}
property_map = {}

In [25]:
with open(path, encoding='utf-8') as f:
    f = json.load(f)['@graph']
    for G in f:
        if G['@type'] == c_t:
            if G['@id'].find(c_xml) != -1:
                class_map[G['@id'].split(c_xml)[-1]] = G['label']['@value']
        elif G['@type'] == p_t:
            if G['@id'].find(p_xml) != -1:
                property_map[G['@id'].split(p_xml)[-1]] = G['label']['@value']
            else:
                property_map[G['@id'].replace(':','')] = G['label']['@value']
        else:
            pass

In [26]:
property_map

{'P0': '名称',
 'P1': '科',
 'P2': '目',
 'P3': '属',
 'P4': '病毒形状',
 'P5': '传播途径',
 'P6': '名称',
 'P7': '医学专科',
 'P8': '发病部位',
 'P9': '临床表现',
 'P10': '常见病因',
 'P11': '传染性',
 'P12': '病原类型',
 'P13': '名称',
 'P14': '科',
 'P15': '界',
 'P16': '属',
 'P17': '名称',
 'P18': '类型',
 'P19': '剂量',
 'P20': '是否处方药',
 'P21': '不良反应',
 'P22': '禁忌',
 'P23': '名称',
 'P24': '别称',
 'P25': '病因',
 'P26': '发病部位',
 'P27': '临床表现',
 'P28': '传染性',
 'P29': '就诊科室',
 'P30': '多发人群',
 'P31': '相关检查',
 'P32': '名称',
 'P33': '学科级数',
 'P34': '学科分支',
 'P35': '常见疾病',
 'P36': '研究领域',
 'P37': '主要检查方法',
 'P38': '主要治疗方法',
 'P39': '中文名',
 'P40': '外文名',
 'P41': '分类',
 'P42': '作用',
 'P43': '正常值',
 'P44': '来源',
 'P45': '来源',
 'P46': '来源',
 'P47': '来源',
 'P48': '来源',
 'P49': '来源',
 'P50': '来源',
 'P51': '引起',
 'P52': '引起',
 'P53': '可医治',
 'P54': '适用科室',
 'P55': '适用症状',
 'P56': '适用疾病',
 'P57': '常用药物',
 'P58': '就诊于',
 'P59': '相关疾病',
 'P60': '临床症状',
 'P61': '主要病因',
 'P62': '常用药物',
 'P63': '就诊科室',
 'P64': '相关症状',
 'P65': 'sameAs',
 'P66': 'sameAs'

In [27]:
entity_id = []
entity_value = []
entity_label = []
rel_from = []
rel_to = []
rel_type = []
dateC = 1
stringC = 1

entity = pd.DataFrame()
rel = pd.DataFrame()

In [28]:
with open(path, encoding='utf-8') as f:
    f = json.load(f)['@graph']
    for G in f:
        if G['@type'].find(c_xml)!= -1:
            r_id = G['@id'].split(r_xml)[-1] # main resource的 RID
            entity_id.append(r_id)
            entity_value.append(G['label']['@value'])
            entity_label.append(class_map[G['@type'].split(c_xml)[-1]])
            for k in G.keys():
                if k.startswith('P'):
                    # 一个PID可能有多个对应的Resource 或者是单独的String/Date
                    if type(G[k]) == str:
                        rel_from.append(r_id)
                        rel_type.append(property_map[k]) # 创建main resource和property
                        if is_date(G[k]) == True: # 若是date字符串则 需要创建entity 并设置RID
                            dr_id = 'dR'+ str(dateC) # Character_Date dR+ID
                            dateC += 1
                            entity_id.append(dr_id)
                            entity_value.append(G[k])
                            entity_label.append('Wiki_Date')
                            rel_to.append(dr_id) # 记得加上rel_to 连接main resource
                            
                        elif G[k].find(r_xml) == -1:
                        # 若是String的值 同样需要创建entity
                            sr_id = 'sR'+ str(stringC) # Character_String sR+ID
                            stringC += 1
                            entity_id.append(sr_id)
                            entity_value.append(G[k])
                            entity_label.append('Wiki_String')
                            rel_to.append(sr_id) # 记得加上rel_to 连接main resource
                        else:
                            rel_to.append(G[k].split(r_xml)[-1])
                    else: # 若不是str那么就是list, 即url_list
                        for url in G[k]:
                            rel_from.append(r_id)
                            rel_type.append(property_map[k]) # 创建main resource和property
                            rel_to.append(url.split(r_xml)[-1])

In [29]:
print(len(entity_id))
print(len(entity_value))
print(len(entity_label))
print(len(rel_from))
print(len(rel_to))
print(len(rel_type))

225108
225108
225108
182517
182517
182517


In [30]:
entity['Wiki:ID(Wiki-ID)'] = entity_id
entity['value'] = entity_value
entity[':LABEL'] = entity_label
rel[':START_ID(Wiki-ID)'] = rel_from
rel[':END_ID(Wiki-ID)'] = rel_to
rel[':TYPE'] = rel_type

In [31]:
entity.to_csv(r'output/wiki_entity.csv', encoding='utf-8', index=False)
rel.to_csv(r'output/wiki_rel.csv', encoding='utf-8',  index=False)

In [ ]:
neo4j-admin import --mode=csv --nodes import\character_entity.csv --nodes import\event_entity.csv  --relationships import\character_rel.csv --relationships import\event_rel.csv
